In [32]:
#Benotigte Madule(inkl. in Anaconda)
import numpy as np
import matplotlib.pyplot as plt
from scipy.odr import *
import random
from scipy.stats import norm

import sys

def gau(x,m,o):
    return (1/np.sqrt(2*np.pi*o**2))*np.exp(-(x-m)**2/(o**2)*2)


def func(B, x):
    [m1,o1,m2,o2,m3,o3]=B
    return gau(x,m1,o1)+gau(x,m2,o2)+gau(x,m3,o3)
    
    

def fit(x,y,x_err,y_err):
    #definerit die zu fittene Funktion,in B sind die Parameter und x ist die unabhängige
    model = Model(func)

    #Überprüft welche Fehler angegeben wurden
    if (np.isnan(x_err).any() and np.isnan(y_err).any()):
        data = RealData(x, y)
    elif np.isnan(x_err).any():
        data = RealData(x, y, sy=y_err)
    elif np.isnan(y_err).any():
        data = RealData(x, y, sx=x_err)
    else:
        data = RealData(x, y, sx=x_err, sy=y_err)
    
    #Führt den Fit aus, beta0 sind die Startwerte

    odr = ODR(data, model,beta0)
    out = odr.run()
    #out.beta beinhaltet die Parameter
    #out.sd_beta beinhaltet die Fehler der Parameter
    return out

#Spannung
U=np.array([600,500])
U_fehler=1
U_err=np.full(len(U),U_fehler)

#Zeit:
t2=np.array([1,1])
t1=np.array([1,1])
t_fehler=0.1
t_err=np.full(len(t1),t_fehler)
t1_err=t2_err=t_err

#Strecke; x ist gemessen durchs Mikroskop; s ist umgerechnet in den Realen wert
x=np.array([1,1])
x_fehler=0.1
x_err=np.full(len(x),x_fehler)

#Position
z=1.875 #zoom
z_err=1.875*0.01
s=x/z*10**(-4)
s_err=np.sqrt((1/z*10**(-4)*x_err)**2+
              (x*10**(-4)*z_err/z**2)**2)

#Geschwindigkeit
v1=s/t1
v1_err=np.sqrt((s_err/t1)**2+(s*t1_err/t1**2)**2)
v2=s/t2
v2_err=np.sqrt((s_err/t2)**2+(s*t2_err/t2**2)**2)


eta=1.81*10**(-5)
d=6*10**(-3)
poel=875.3
pl=1.29
p=875
g=9.81

Q=(v1+v2)*np.sqrt(v1)/U*eta**(3/2)*18*np.pi*d/np.sqrt(2*p*g)
#Als BSP
Q=np.array([1,2,3,3,3,3,3,4,6,6,6,6,6,8,9,9,9,9,10])*10**(-19);


Q_err=9*np.pi*d*eta**(3/2)/np.sqrt(g*p)*np.sqrt(((3*v1+v2)/(U*np.sqrt(2*v1))*v1_err)**2+
                                                (np.sqrt(2*v1)/U*v2_err)**2+
                                                (np.sqrt(2*v1)*(v1+v2)/U**2*U_err)**2)
#Als BSP:
Q_err=np.nan




#Anpassen für schönnes Histogramm
stepsize=max(Q)/10

#Histogramm erstellen
bins=np.arange(min(Q)-stepsize/2,max(Q)+stepsize/1.5,stepsize)
#[n,binsh,ka]=plt.hist(Q,bins=bins,density=False,edgecolor='black', linewidth=1.2)

plt.figure()

norm=np.linalg.norm(n)
n=n/norm
#Fit erstellen
xf=np.arange(min(Q),max(Q)+stepsize,stepsize)
yf=n

out=fit(xf,yf,Q_err,np.nan)

#Fit-Funktionswerte Berechnen
x_p=np.linspace(min(Q),max(Q),1000)
y_p=func(out.beta,x_p)

#Plotten
f=plt.plot(x_p,y_p)
plt.grid()

print("mu1/sigma1/mu2/sigma2/mu3/sigma3:")
print(out.beta)
print("Fehler:")
print(out.sd_beta)

print("Q:")
print(Q)
print("n:(y fit)")
print(n)
print("bins:")
print(bins)
print("x fit")
print(xf)

ValueError: not enough values to unpack (expected 9, got 3)

<Figure size 432x288 with 0 Axes>

In [33]:
beta0

[array([1.e+00, 3.e-19, 1.e-19]),
 array([1.e+00, 6.e-19, 1.e-19]),
 array([1.e+00, 6.e-19, 1.e-19])]